In [99]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [100]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [101]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [102]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [103]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

# Normalize the data
scaled_data = scaler.fit_transform(df_market_data)



In [104]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled_data, columns=df_market_data.columns, index=df_market_data.index)

# Copy the crypto names from the original data
crypto_names = df_market_data.index

# Set the 'coinid' column as index
scaled_df.index = crypto_names


# Display sample data
print("Sample Scaled Data:")
print(scaled_df.head())


Sample Scaled Data:
              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash            

---

### Find the Best Value for k Using the Original Data.

In [105]:
# Create a list with the number of k-values from 1 to 11
k_values = range(1, 12)

In [106]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in range(1, 12):
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # 2. Fit the model to the data using df_market_data_scaled
    kmeans.fit(scaled_df)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

# Print the inertia values
print("Inertia Values:")
print(inertia_values)


c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environme

Inertia Values:
[287.0, 195.82021818036043, 123.19048183836959, 79.02243535120975, 65.22036416927935, 52.8298185338292, 43.78643495414179, 37.527739610654386, 33.07092554993108, 28.989073283679502, 24.532527356516862]


In [107]:
# Create a dictionary with the data to plot the Elbow curve
inertia_dict = {
    'Number of Clusters (k)': range(1, 12),
    'Inertia': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
inertia_df = pd.DataFrame(inertia_dict)
print("Data for Elbow Curve:")
print(inertia_df)

Data for Elbow Curve:
    Number of Clusters (k)     Inertia
0                        1  287.000000
1                        2  195.820218
2                        3  123.190482
3                        4   79.022435
4                        5   65.220364
5                        6   52.829819
6                        7   43.786435
7                        8   37.527740
8                        9   33.070926
9                       10   28.989073
10                      11   24.532527


In [108]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
inertia_df.hvplot.line(x='Number of Clusters (k)', y='Inertia', title='Elbow Method for Optimal k', xlabel='Number of Clusters (k)', ylabel='Inertia')

:Curve   [Number of Clusters (k)]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [109]:
# Initialize the K-Means model using the best value for k
best_k = 4

kmeans_model = KMeans(n_clusters=best_k, random_state=42)

In [110]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(scaled_data)

c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=42)

In [111]:
# Predict cluster labels for the scaled data
predicted_clusters = kmeans_model.predict(scaled_data)

# Print the resulting array of cluster values
print("Predicted Clusters:")
print(predicted_clusters)


Predicted Clusters:
[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [112]:
# Create a copy of the DataFrame
df_market_data_copy = df_market_data.copy()

In [113]:
# Add the predicted clusters as a new column to the copied DataFrame
df_market_data_copy['Cluster'] = predicted_clusters

# Display sample data with the new 'Cluster' column
print("Sample Data with Predicted Clusters:")
print(df_market_data_copy.head())


Sample Data with Predicted Clusters:
              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                           1.08388                     7.60278   
ethereum                          0.22392                    10.38134   
tether                           -0.21173                     0.04935   
ripple                           -0.37819                    -0.60926   
bitcoin-cash                      2.90585                    17.09717   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                           6.57509                      7.67258   
ethereum                          4.80849                      0.13169   
tether                            0.00640                     -0.04237   
ripple                            2.24984                      0.23455   
bitcoin

In [114]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot = df_market_data_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",  # Color the points based on the 'Cluster' column
    hover_cols=["coin_id"],  # Display the cryptocurrency name on hover
    height=500,
    width=800,
    title="Cryptocurrency Clusters",
    xlabel="Price Change Percentage (24h)",
    ylabel="Price Change Percentage (7d)",
    cmap="Category10",  # Use a categorical color map
)

scatter_plot

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [115]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [116]:
# Use fit_transform to reduce to three principal components
pca_transformed_data = pca_model.fit_transform(df_market_data_copy.drop('Cluster', axis=1))

# Create a DataFrame with the transformed data
pca_df = pd.DataFrame(data=pca_transformed_data, columns=['PC1', 'PC2', 'PC3'], index=df_market_data_copy.index)

# View the first five rows of the DataFrame
pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-341.800963,-51.366775,12.525471
ethereum,-249.420466,24.117548,-14.231466
tether,-402.614721,-118.710737,24.838397
ripple,-406.752437,-79.487286,1.566331
bitcoin-cash,-382.429948,-103.431959,16.753073


In [117]:
# Retrieve the explained variance
explained_variance = pca_model.explained_variance_ratio_

# Display the explained variance for each principal component
print("Explained Variance:")
for i, ev in enumerate(explained_variance):
    print(f"PC{i + 1}: {ev:.2%}")

Explained Variance:
PC1: 97.60%
PC2: 2.30%
PC3: 0.07%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 100%

In [118]:
# Creating a DataFrame with the PCA data
pca_df = pd.DataFrame(data=pca_transformed_data, columns=['PC1', 'PC2', 'PC3'], index=df_market_data_copy.index)

# Copy the crypto names from the original data
pca_df['coin_id'] = df_market_data_copy.index

# Set the coinid column as the index
pca_df.set_index('coin_id', inplace=True)

# Display sample data
print("Sample PCA Data:")
print(pca_df.head())


Sample PCA Data:
                     PC1         PC2        PC3
coin_id                                        
bitcoin      -341.800963  -51.366775  12.525471
ethereum     -249.420466   24.117548 -14.231466
tether       -402.614721 -118.710737  24.838397
ripple       -406.752437  -79.487286   1.566331
bitcoin-cash -382.429948 -103.431959  16.753073


---

### Find the Best Value for k Using the PCA Data

In [119]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [120]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Create a KMeans model using the loop counter for the n_clusters
    kmeans_model_pca = KMeans(n_clusters=k, random_state=1)
    
    # Fit the model to the data using df_market_data_pca
    kmeans_model_pca.fit(pca_df)
    
    # Append the model.inertia_ to the inertia list
    inertia_values_pca.append(kmeans_model_pca.inertia_)


c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environme

In [121]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k_values": k_values,
    "inertia": inertia_values_pca
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df_pca = pd.DataFrame(elbow_data_pca)
elbow_df_pca


,k_values,inertia
0,1,6.997052e+07
1,2,8.180192e+06
2,3,2.580721e+06
3,4,8.237471e+05
4,5,4.264175e+05
5,6,2.592098e+05
6,7,1.897981e+05
7,8,1.472967e+05
8,9,1.086863e+05
9,10,8.556361e+04


In [122]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df_pca.hvplot.line(
    x='k_values',
    y='inertia',
    title='PCA Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    line_width=2
)

:Curve   [k_values]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [123]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=42)

In [130]:
# Fit the K-Means model using the PCA data
kmeans.fit(pca_df)

c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\crowl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=42)

In [125]:
# Predict the clusters for the PCA data
pca_clusters = kmeans.predict(pca_df)

# Print the resulting array of cluster values
print("Predicted Clusters (PCA Data):")
print(pca_clusters)


Predicted Clusters (PCA Data):
[0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 1 0 3 3 2
 0 0 0 3]


In [126]:
# Create a copy of the DataFrame with the PCA data
df_pca_clusters = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_clusters['Cluster'] = kmeans_model_pca.labels_

# Display sample data
print("Sample Data with Predicted Clusters (PCA Data):")
print(df_pca_clusters.head())


Sample Data with Predicted Clusters (PCA Data):
                     PC1         PC2        PC3  Cluster
coin_id                                                 
bitcoin      -341.800963  -51.366775  12.525471        0
ethereum     -249.420466   24.117548 -14.231466        6
tether       -402.614721 -118.710737  24.838397        8
ripple       -406.752437  -79.487286   1.566331        8
bitcoin-cash -382.429948 -103.431959  16.753073        8


In [127]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"],
    width=800,
    height=600,
    title="PCA - Cryptocurrency Clusters",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2"
)


:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [128]:
# Composite plot to contrast the Elbow curves using hvplot
elbow_df_pca.hvplot.line(x='k_values',y='inertia',title='PCA Elbow Curve',xlabel='Number of Clusters (k)',ylabel='Inertia',line_width=2
) + inertia_df.hvplot.line(x='Number of Clusters (k)', y='Inertia', title='Elbow Method for Optimal k', xlabel='Number of Clusters (k)', ylabel='Inertia')

:Layout
   .Curve.I  :Curve   [k_values]   (inertia)
   .Curve.II :Curve   [Number of Clusters (k)]   (Inertia)

In [129]:
# Composite plot to contrast the clusters
df_pca_clusters.hvplot.scatter(x="PC1",y="PC2",by="Cluster",hover_cols=["coin_id"],width=800,height=600,title="PCA - Cryptocurrency Clusters",xlabel="Principal Component 1",ylabel="Principal Component 2"
) + df_market_data_copy.hvplot.scatter(x="price_change_percentage_24h",y="price_change_percentage_7d",by="Cluster",  width=800,height = 600, title="Cryptocurrency Clusters",xlabel="Price Change Percentage (24h)",ylabel="Price Change Percentage (7d)",cmap="Category10",  
)

:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** While I used the same value for k for both, the difference in the predicted cluster results highlights the difference in using fewer features. The overall structure of the clusters is somewhat consistent, there is variation in the assignment of data points. This suggests that the reduced features captures essential information for clustering, but also introduces some differences in their assignments. 